# Capability Host troubleshooting for AI Foundry (project-based)
This notebook runs the HTTP requests provided in the `test.http` file using Python.
Create .env file in the notebook directory, based on `.env.example` file

In [ ]:
# Install required packages

!pip install requests azure.identity python-dotenv

In [ ]:
import requests
import json
import os
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(verbose=True, override=True)

project_resource_id = os.environ.get("projectResourceId", None)
subnet_id = os.environ.get("subnetId", None)

if project_resource_id is None:
    raise ValueError("❌ projectResourceId environment variable is not set. 🔧 Please set it in your .env file.")

# Parse project_resource_id to extract variables
# Format: /subscriptions/{subscriptionId}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{foundryName}/projects/{projectName}
parts = project_resource_id.split("/")
try:
    subscription_id = parts[parts.index("subscriptions") + 1]
    rg = parts[parts.index("resourceGroups") + 1]
    foundry_name = parts[parts.index("accounts") + 1]
    project_name = parts[parts.index("projects") + 1]
except (ValueError, IndexError) as e:
    raise ValueError("❌ Invalid projectResourceId format. 📝 Expected format: /subscriptions/{{subscriptionId}}/resourceGroups/{{rg}}/providers/Microsoft.CognitiveServices/accounts/{{foundryName}}/projects/{{projectName}}") from e

print(f"✅ Parsed project resource ID: {project_resource_id[:50]}...")
print(f"   📦 Subscription: {subscription_id}")
print(f"   📁 Resource Group: {rg}")
print(f"   🏭 Foundry: {foundry_name}")
print(f"   📋 Project: {project_name}")

print("🔧Azure CLI commands to get capability hosts:")
print("   Get capability hosts:")
print(f"     🔧az rest --method get --url https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{foundry_name}/projects/{project_name}/capabilityHosts?api-version=2025-04-01-preview")
print("   Delete account capability host:")
print(f"     🔧az rest --method delete --url https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{foundry_name}/projects/{project_name}/capabilityHosts/{foundry_name}@aml_aiagentservice?api-version=2025-04-01-preview")
print("   Create account capability host:")
print(f"     🔧az rest --method put --url https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{foundry_name}/projects/{project_name}/capabilityHosts/{foundry_name}caphost?api-version=2025-04-01-preview --body '{{\"properties\": {{\"customerSubnet\": \"{subnet_id}\", \"capabilityHostKind\": \"Agents\"}}}}'") 
print("   Get project capability hosts:")
print(f"     🔧az rest --method get --url https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{foundry_name}/projects/{project_name}/capabilityHosts?api-version=2025-04-01-preview")

tenant_id = os.environ.get("tenantId", None)
base_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/"

# Get token using DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token(
    "https://management.azure.com/.default", tenant_id=tenant_id
)

# Define headers
headers = {"Authorization": f"Bearer {token.token}", "Content-Type": "application/json"}

### Get Account Capability Host

In [ ]:
url = f"{base_url}{foundry_name}/capabilityHosts/?api-version=2025-06-01"
response = requests.get(url, headers=headers)
response_json = response.json()
found_subnet = response_json['value'][0]['properties']['customerSubnet'] if 'value' in response_json and len(response_json['value']) > 0 else None
if found_subnet:
    print(
        f"✅ Foundry using subnet: 🔧{found_subnet}🔧"
    )
else:
    print("❌ No subnet found. Foundry is NOT VNET INJECTED.")
    
print(json.dumps(response_json, indent=4))


### Create Account Capability Host

In [ ]:
url = f"{base_url}{foundry_name}/capabilityHosts/{foundry_name}caphost?api-version=2025-06-01"
payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "customerSubnet": "<subnet_resource_id>",  # Replace with actual subnet resource ID
    }
}
# print(json.dumps(payload, indent=4))
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Delete Account Capability Host
Capability host needs to be deleted if it's in **Failed** state. Project capability needs to be deleted before hub capability.

In [ ]:
url = f"{base_url}{foundry_name}/capabilityHosts/{foundry_name}@aml_aiagentservice?api-version=2025-06-01"
# avoid incidentally deleting the capability host
user_input = input("Type YES to continue with deletion or Ctrl+C to abort...")
if user_input == "YES":
    response = requests.delete(url, headers=headers)
    print(response.status_code)

### Create Project Capability Host

In [ ]:
url = f"{base_url}{foundry_name}/projects/{project_name}/capabilityHosts/projcaphost?api-version=2025-06-01"
payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "vectorStoreConnections": ["srch-mfgai-p-naa-007-for-ai-project-1"],
        "storageConnections": ["stmfgaipnaa007-for-ai-project-1"],
        "threadStorageConnections": ["cosmos-mfgai-p-naa-007-for-ai-project-1"],
    }
}
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Get Project Capability Host

In [ ]:
url = f"{base_url}{foundry_name}/projects/{project_name}/capabilityHosts?api-version=2025-06-01"
print(url)  # For debugging purposes
response = requests.get(url, headers=headers)
print(response.status_code)
print(json.dumps(response.json(), indent=4))

### Delete Project Capability Host

In [ ]:
url = f"{base_url}{foundry_name}/projects/{project_name}/capabilityHosts/projcaphost?api-version=2025-06-01"
# avoid incidentally deleting the capability host
user_input = input("Type YES to continue with deletion or Ctrl+C to abort...")
if user_input == "YES":
    response = requests.delete(url, headers=headers)
    print(response.status_code)

### POST to Azure OpenAI Service

In [ ]:
ai_token = credential.get_token("https://ai.azure.com")
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + ai_token.token,
}

# get deployments
url = f"https://{foundry_name}.services.ai.azure.com/api/projects/{project_name}/deployments?api-version=v1"
response = requests.get(url, headers=headers)

# grab the first deployment name
deployment_name = response.json().get("value", [{}])[0].get("name", None)
print(f"Deployment name: {deployment_name}")
# print(json.dumps(response.json(), indent=4))

url = f"https://{foundry_name}.services.ai.azure.com/models/chat/completions?api-version=2024-05-01-preview"

payload = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {
            "role": "assistant",
            "content": "Yes, customer managed keys are supported by Azure OpenAI.",
        },
        {"role": "user", "content": "Do other Azure AI services support this too?"},
    ],
    "model": deployment_name,
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

# Connections

In [ ]:
url = f"{base_url}{foundry_name}/projects/{project_name}/connections?api-version=2025-06-01"
print(url)  # For debugging purposes
response = requests.get(url, headers=headers)
response_json = response.json()

for connection in response_json.get("value", []):
    print(f"Connection ID: {connection.get('id')}")
    print(f"Connection Name: {connection.get('name')}")
    print(f"Connection Category: {connection.get('properties', {}).get('category')}")
    print("-" * 40)

print(json.dumps(response_json, indent=4))

In [ ]:
# Update connection example
connection_name = "model-gateway-bro6ttlxdau5e-static"  # Replace with actual connection name
url = f"{base_url}{foundry_name}/projects/{project_name}/connections/{connection_name}?api-version=2025-06-01"
key = input(f"Enter the API key for the connection {connection_name}: ")
payload = {
    "name": connection_name,
    "type": "Microsoft.CognitiveServices/accounts/projects/connections",
    "properties": {
        "authType": "ApiKey",
        "credentials": {
            "key": key
        },
        "group": "AzureAI",
        "category": "ModelGateway",
        "target": "https://apim-bro6ttlxdau5e.azure-api.net/inference/openai",
        "isSharedToAll": True,
        "metadata": {
            "deploymentInPath": "false",
            "inferenceAPIVersion": "2025-03-01-preview",
            "models": "[{\"name\":\"gpt-4.1-mini\",\"properties\":{\"model\":{\"name\":\"gpt-4.1-mini\",\"version\":\"2025-01-01-preview\",\"format\":\"OpenAI\"}}},{\"name\":\"gpt-5-mini\",\"properties\":{\"model\":{\"name\":\"gpt-5-mini\",\"version\":\"2025-04-01-preview\",\"format\":\"OpenAI\"}}},{\"name\":\"o3-mini\",\"properties\":{\"model\":{\"name\":\"o3-mini\",\"version\":\"2025-01-01-preview\",\"format\":\"OpenAI\"}}}]"
        }
    }
}
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

## Get Agents

In [ ]:
url = f"https://{foundry_name}.services.ai.azure.com/api/projects/{project_name}/assistants?api-version=v1"
print(url)  # For debugging purposes
token = credential.get_token("https://ai.azure.com/.default", tenant_id=tenant_id)
headers = {"Authorization": f"Bearer {token.token}"}

response = requests.get(url, headers=headers)
response_json = response.json()
print(json.dumps(response_json, indent=4))